In [257]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
import os.path

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/different-hair-types/04-Kinky/frame00369.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00400.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00366.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00402.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00463.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00399.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00401.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00466.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00460.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00387.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00374.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00424.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00438.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00449.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00469.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00406.jpg
/kaggle/input/different-hair-types/04-Kinky/frame00373.j

In [258]:
image_dir = Path('../input/different-hair-types')

# Create File DataFrame

In [259]:
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name = 'Filepath').astype(str)
labels = pd.Series(labels, name= 'Label')

image_df = pd.concat([filepaths, labels], axis = 1)

In [260]:
image_df

,Filepath,Label
0,../input/different-hair-types/04-Kinky/frame00...,04-Kinky
1,../input/different-hair-types/04-Kinky/frame00...,04-Kinky
2,../input/different-hair-types/04-Kinky/frame00...,04-Kinky
3,../input/different-hair-types/04-Kinky/frame00...,04-Kinky
4,../input/different-hair-types/04-Kinky/frame00...,04-Kinky
...,...,...
835,../input/different-hair-types/05-Braids/frame0...,05-Braids
836,../input/different-hair-types/05-Braids/frame0...,05-Braids
837,../input/different-hair-types/05-Braids/frame0...,05-Braids
838,../input/different-hair-types/05-Braids/frame0...,05-Braids


In [261]:
train_df, test_df = train_test_split(image_df, train_size = 0.7, shuffle = True, random_state = 1)

In [262]:
train_df

,Filepath,Label
40,../input/different-hair-types/04-Kinky/frame00...,04-Kinky
89,../input/different-hair-types/04-Kinky/frame00...,04-Kinky
487,../input/different-hair-types/02-Wavy/frame001...,02-Wavy
704,../input/different-hair-types/07-Short-men/fra...,07-Short-men
68,../input/different-hair-types/04-Kinky/frame00...,04-Kinky
...,...,...
715,../input/different-hair-types/07-Short-men/fra...,07-Short-men
767,../input/different-hair-types/05-Braids/frame0...,05-Braids
72,../input/different-hair-types/04-Kinky/frame00...,04-Kinky
235,../input/different-hair-types/01-Straight/fram...,01-Straight


# Load Image Data

In [263]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
rescale = 1. / 255,
    horizontal_flip = True,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    validation_split = 0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
rescale = 1. / 255
)



In [264]:
train_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    #class_mode = 'raw',
    batch_size = 32,
    shuffle = True,
    seed = 42,
    subset = 'training'
    
)

val_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    #class_mode = 'raw',
    batch_size = 32,
    shuffle = True,
    seed = 42,
    subset = 'validation'
    
)

test_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'Filepath',
    y_col = 'Label',
    target_size = (224, 224),
    color_mode = 'rgb',
    #class_mode = 'raw',
    batch_size = 32,
    shuffle = False,
 
    
)

Found 471 validated image filenames belonging to 7 classes.
Found 117 validated image filenames belonging to 7 classes.
Found 588 validated image filenames belonging to 7 classes.


# Training

In [265]:
inputs = tf.keras.Input(shape=(120,120, 7))


x = tf.keras.layers.Conv2D(filters = 14, kernel_size = (3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters = 28, kernel_size=(3, 3), activation = 'relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation = 'relu')(x)
x = tf.keras.layers.Dense(128, activation = 'relu')(x)

outputs= tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
   metrics = ['accuracy']
)

history = model.fit(
    train_images,
    validation_data = val_images,
    epochs = 100,
   callbacks= 
    [
        tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience = 5,
        restore_best_weights = True
        ),
       
        tf.keras.callbacks.ReduceLROnPlateau(
        monitor = 'val_loss',
        patience = 3
            ) 
              
       ]
    )


Epoch 1/100


InvalidArgumentError:  input depth must be evenly divisible by filter depth: 3 vs 7
	 [[node model_22/conv2d_52/Relu (defined at <ipython-input-265-3b993b9a8d05>:36) ]] [Op:__inference_train_function_16239]

Function call stack:
train_function


* 

In [ ]:
#train_images.class_indices

# Results

In [ ]:
results = model.evaluate(test_images, verbose = 0)

print("Test Loss: {:.5f}".format(results[0]))

print("Test Accuracy: {:.2f}".format(results[1] * 100))



In [ ]:
predictions = model.predict(test_images) >= 0.5
